In [671]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import joblib

%matplotlib inline 

In [672]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [673]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        low.append(1 + 14*random.random())
        up.append(1 + 14*random.random())
        if low[i] > up[i]:
            temp = low[i]
            low[i] = up[i]
            up[i] = temp
        
        lb = lb + b[i]*low[i];
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = lb + (ub - lb)*0.7;
    
    return CqkProblem( r, n, d, a, b, low, up)

In [674]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [214]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [215]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    derivs = []
    phis = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    
    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        sum_slopes = sum(slopes)
        
    if phi == 0.0:
        return it,lambs,derivs,phis,sum_slopes
    elif alfa == beta:
        return -1,lambs,derivs,phis,sum_slopes
    else:
        return -2,lambs,derivs,phis,sum_slopes

In [406]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def new_newton(p,slopes,predicted_lamb):
    lambs = [] 
    derivs = []
    phis = []
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    x = []
    r = p.r
    
    lamb = predicted_lamb
    deriv = 26525.250920050636
    phi = -1
    
#     if phi > 0:
#         lambda_n = lamb - (phi/deriv)
#         if abs(lambda_n - lamb) <= 0.00000000001:
#             phi = 0.0
#             break
#         if lambda_n > alfa:
#             lamb = lambda_n
#         else:
            
                    
#     deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)

    it = 0
    
    test_lamb = 0
    while phi != 0.0 and it <= MAX_IT:
        if it == 1:
            test_lamb = lamb
        if it == 2 and test_lamb == lamb:
            lambs = [] 
            derivs = []
            phis = []
            it = 0
            phi = 1
            alfa = INFINITO_NEGATIVO;
            beta = INFINITO_POSITIVO;
    
        if phi > 0:
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
#                     if it==0:
#                         lamb = predicted_lamb
#                         phi = -1
#                         beta = INFINITO_POSITIVO
#                         continue
                else:
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        sum_slopes = sum(slopes)
        
    if phi == 0.0:
        return it,lambs,derivs,phis,sum_slopes
    elif alfa == beta:
        return -1,lambs,derivs,phis,sum_slopes
    else:
        return -2,lambs,derivs,phis,sum_slopes

In [881]:
n = 1000
p = generate_cqk_problem(n)

In [882]:
%%time

it,lambs, derivs,phis,sum_slopes = newton(p)

it, lambs, derivs, phis

CPU times: user 12 ms, sys: 1.5 ms, total: 13.5 ms
Wall time: 13.6 ms


(5,
 [7.152557711326608,
  8.617685833458852,
  8.745144732884834,
  8.747026397280251,
  8.74702825429127],
 [6575.101260182665,
  5531.408987463026,
  5446.75038555087,
  5433.619546450492,
  5433.619546450492],
 [-9633.365762160776,
  -705.027301817021,
  -10.248956271221104,
  -0.010090291367617965,
  4.416733645484783e-11])

In [771]:
def phi_lambda_predicted(p,lamb,r):
    deriv = 0.0
    phi = r * -1
    x = []
    sum_slope = 0
    slopes = []
    for i in range(p.n):
        slope = (p.b[i]/p.d[i])*p.b[i]
        sum_slope = sum_slope + slope
        slopes.append(slope)
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slope
        phi = phi + p.b[i] * x[i];
    return deriv, phi,sum_slope, slopes

In [877]:
# Padronização
scaler = StandardScaler()
scaler_sub_final = StandardScaler()
def test(p):
    som_a = 0
    som_b = 0
    som_d = 0
    
    s0=0.0
    q0=0.0
    for i in range(p.n):
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
        som_a = som_a + p.a[i]
        som_b = som_b + p.b[i]
        som_d = som_d + p.d[i]
    first_lamb = (p.r-s0)/q0
    

    new_a = som_a/p.n
    new_b = som_b/p.n
    new_d = som_d/p.n
    rn = p.r/p.n
    

#     scaler=joblib.load('scaler_initial_lamb.bin')
#     x_values_lamb_init = scaler.transform(np.asarray([[new_a, new_b, new_d,rn ]]))
#     loaded_model_initial_lamb = joblib.load('finalized_model_initial_lamb.sav')
#     first_lamb = loaded_model_initial_lamb.predict(x_values_lamb_init)[0]

    first_deriv, first_phi, sum_slope,slopes = phi_lambda_predicted(p,first_lamb,p.r)
    
    scaler_sub_final=joblib.load('scaler_second_lamb.bin')
    x_values_lamb_final = scaler_sub_final.transform(np.asarray([[new_a, new_b, new_d,rn,first_lamb,first_phi, first_deriv, sum_slope]]))
    
    loaded_model_sub_final_lamb = joblib.load('finalized_model_second_lamb.sav')
    predicted_lamb = loaded_model_sub_final_lamb.predict(x_values_lamb_final)[0]
    print(predicted_lamb)
    scaler_third=joblib.load('scaler_third_lamb.bin')
    x_values_lamb_final = scaler_third.transform(np.asarray([[new_a, new_b, new_d,rn,first_lamb,first_phi, first_deriv, sum_slope,predicted_lamb]]))
    
    loaded_model_third_lamb = joblib.load('finalized_model_third_lamb.sav')
    predicted_lamb = loaded_model_third_lamb.predict(x_values_lamb_final)[0]
    print(predicted_lamb)
    return predicted_lamb, new_newton(p,slopes,predicted_lamb)

In [883]:
test(p)

8.6296906520784
8.787844894018647


(8.787844894018647,
 (3,
  [8.787698498183637, 8.747072445341068, 8.747028254291285],
  [5431.955596719649, 5433.619546450492, 5433.619546450492],
  [220.67891511081876, 0.24011735187973215, 1.3248779850982828e-10],
  18766.527617882908))

In [852]:
lista = []
lista2 = []
for i in range(10000):
    n = 1000
    p = generate_cqk_problem(n)
    it,lambs, derivs,phis,sum_slopes = newton(p)
    soma_a = 0
    soma_b = 0
    soma_low = 0
    soma_d = 0
    soma_up = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_low += p.low[i]
        soma_d += p.d[i]
        soma_up += p.up[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_low = soma_low/n
    soma_d = soma_d/n
    soma_up = soma_up/n
    r = p.r/n
    deriv = derivs[0]/n
    
    predicted_lamb, result = test(p)
    
    if it > 3 and it < 6 and predicted_lamb < lambs[-1]:
        l_rs = [soma_a, soma_b, soma_d, r,lambs[0],phis[0],derivs[0],sum_slopes,lambs[1],predicted_lamb,lambs[3],phis[1],derivs[1],lambs[-1]]

        lista.append(l_rs)
    
    

In [853]:

dd = lista

feature_names = ['a','b','d','r','initial_lamb','init_phi','init_deriv','sum_slopes','second_lambda','predicted','fourth_lamb','second_phi','second_deriv','final_lamb']





In [854]:
knapsack_ = {"data":dd, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack_['data'], columns = knapsack_['feature_names'])

In [865]:
# Coletando x e y

X = dataset.iloc[:,:-5]
y = dataset['final_lamb'].values

In [866]:
X

,a,b,d,r,initial_lamb,init_phi,init_deriv,sum_slopes,second_lambda
0,17.659,17.004467,17.382,152.618323,7.084457,-10016.259038,6720.539154,18915.242444,8.574852
1,17.367,16.729544,17.438,150.137728,7.186086,-10144.105143,6997.709847,18388.813563,8.635718
2,17.436,16.666572,17.342,149.794038,7.189541,-11129.923426,6335.057874,18322.578554,8.946419
3,17.441,17.067416,17.494,155.459953,7.229492,-9162.137440,7042.093034,18972.226425,8.530545
4,17.621,17.054237,17.611,151.144904,7.012699,-11155.385904,6927.600687,18901.215556,8.622980
...,...,...,...,...,...,...,...,...,...
6101,17.630,16.844900,17.563,148.651253,7.065885,-10950.483140,6329.932445,18450.872098,8.795837
6102,17.709,16.974108,17.352,153.102523,7.104306,-10973.334376,7288.246086,18918.383978,8.609927
6103,17.584,16.837233,17.568,149.365356,7.124032,-10731.066419,6758.794382,18415.928446,8.711751
6104,17.561,17.067261,17.275,151.580633,6.929345,-10790.459403,7436.771293,19192.809990,8.380305


In [867]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [868]:
# Padronização
scaler = StandardScaler()
scaler.fit(X_train)

# Aplicando a padronização aos dados
X_train_p = scaler.transform(X_train)
X_test_p = scaler.transform(X_test)

In [869]:
joblib.dump(scaler, 'scaler_final_lamb.bin', compress=True)

['scaler_final_lamb.bin']

In [870]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr = MLPRegressor(activation='logistic',random_state=1, max_iter=10000).fit(X_train_p, y_train)

In [871]:
regr.fit(X_train_p, y_train)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [872]:
finalized_model = 'finalized_model_final_lamb.sav'
joblib.dump(regr, finalized_model)

['finalized_model_final_lamb.sav']

In [873]:
loaded_model = joblib.load(finalized_model)

In [874]:
regr.score(X_test_p, y_test)

0.9769374695268368